# Skripsi
# Analisis sentimen menggunakan NBC

#### I Komang Damai Armawan

In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import re
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import classification_report 
from collections import Counter
from imblearn.over_sampling import SMOTE
# from IPython import display
# import matplotlib.pyplot as plt

# A. Read Dataset

In [2]:
df = pd.read_excel(r"D:\work\project\SentimentAnalysis-MyPertamina\data\dataset twt - edited.xlsx")
df.head(5)

,url,created_at,full_text,reply_count,retweet_count,user/advertiser_account_type,user/description,user/followers_count,user/friends_count,user/location,user/media_count,user/id,user/name,user/screen_name,user/url,user/verified
0,https://twitter.com/Cutty__Flam/status/1595649...,2022-11-24T05:24:20.000Z,@xa_adinn @RossaGhea @officialJKT48 ribet ga w...,1,0,promotable_user,"ARSENAL, LAZIO, T-ara, One Piece, and JKT48 lo...",57,254,Kemanggisan,1100,1246673035,Furanky,Cutty__Flam,NaN,False
1,https://twitter.com/jawapos/status/15962776895...,2022-11-25T23:00:33.000Z,"17 Desember, Pembelian BBM Subsidi di SPBU Waj...",0,0,promotable_user,"Akun Twitter resmi Jawa Pos Media | Jawa Pos, ...",512472,267,Jakarta,19383,2392460208,Jawa Pos,jawapos,https://t.co/zQIlbGPP2B,True
2,https://twitter.com/wahananewsdotco/status/159...,2022-11-26T01:24:33.000Z,"Data Konsumen Diambil Saat Isi BBM Subsidi, In...",0,2,promotable_user,Akun Resmi Media Wahana News - Inspirasi Konsu...,4837,1924,Jakarta,79,1124517245722349568,Wahana News Official,wahananewsdotco,https://t.co/CHwYCTxjr6,False
3,https://twitter.com/linkaja/status/15963681492...,2022-11-26T05:00:00.000Z,Isi BBM lebih banyak biar makin irit!\nBeli BB...,2,0,promotable_user,Layanan Customer Service LinkAja: Live Chat ht...,117575,13,NaN,10380,3195998996,LinkAja,linkaja,https://t.co/IfRujVkEqz,True
4,https://twitter.com/SyariahLinkAja/status/1596...,2022-11-27T11:00:02.000Z,"Isi BBM via MyPertamina, beli 20L dapat ekstra...",0,0,promotable_user,LinkAja Syariah adalah Aplikasi Dompet Elektro...,1906,4,NaN,3279,1297815347404263424,LinkAja Syariah,SyariahLinkAja,https://t.co/OgZKSEz68B,True


# B. Preprocessing
##### 1.Drop Duplicates 2.User Selection 3.Numbers Removal 4.Punctuation Removal  5.Whitespace Remover
##### 6.Stemming 7.Words Dictionary 8.Tokenizing 9.Stopwords Removal 10.Labeling

In [3]:
# fill the NaN 
df.fillna(0, inplace=True)
df.head(5)

,url,created_at,full_text,reply_count,retweet_count,user/advertiser_account_type,user/description,user/followers_count,user/friends_count,user/location,user/media_count,user/id,user/name,user/screen_name,user/url,user/verified
0,https://twitter.com/Cutty__Flam/status/1595649...,2022-11-24T05:24:20.000Z,@xa_adinn @RossaGhea @officialJKT48 ribet ga w...,1,0,promotable_user,"ARSENAL, LAZIO, T-ara, One Piece, and JKT48 lo...",57,254,Kemanggisan,1100,1246673035,Furanky,Cutty__Flam,0,False
1,https://twitter.com/jawapos/status/15962776895...,2022-11-25T23:00:33.000Z,"17 Desember, Pembelian BBM Subsidi di SPBU Waj...",0,0,promotable_user,"Akun Twitter resmi Jawa Pos Media | Jawa Pos, ...",512472,267,Jakarta,19383,2392460208,Jawa Pos,jawapos,https://t.co/zQIlbGPP2B,True
2,https://twitter.com/wahananewsdotco/status/159...,2022-11-26T01:24:33.000Z,"Data Konsumen Diambil Saat Isi BBM Subsidi, In...",0,2,promotable_user,Akun Resmi Media Wahana News - Inspirasi Konsu...,4837,1924,Jakarta,79,1124517245722349568,Wahana News Official,wahananewsdotco,https://t.co/CHwYCTxjr6,False
3,https://twitter.com/linkaja/status/15963681492...,2022-11-26T05:00:00.000Z,Isi BBM lebih banyak biar makin irit!\nBeli BB...,2,0,promotable_user,Layanan Customer Service LinkAja: Live Chat ht...,117575,13,0,10380,3195998996,LinkAja,linkaja,https://t.co/IfRujVkEqz,True
4,https://twitter.com/SyariahLinkAja/status/1596...,2022-11-27T11:00:02.000Z,"Isi BBM via MyPertamina, beli 20L dapat ekstra...",0,0,promotable_user,LinkAja Syariah adalah Aplikasi Dompet Elektro...,1906,4,0,3279,1297815347404263424,LinkAja Syariah,SyariahLinkAja,https://t.co/OgZKSEz68B,True


In [4]:
df.shape

(972, 16)

### 1. Droping Dataset

In [5]:
# drop duplicates
df.drop_duplicates(subset=['url'])
df.shape

(972, 16)

In [6]:
# user selection
df = df[df['user/name']!='MyPertamina']
df = df[df['user/name']!='LinkAja Syariah']
df = df[df['user/name']!='LinkAja']
df = df[df['user/name']!='Pertamina Papua Maluku']
df = df[df['user/name']!='Pertamax Series ID']
df.shape

(854, 16)

### cut off time date

In [7]:
# fixing the date format
df['created_at'] = pd.to_datetime(df['created_at'])
df['created_at'] = df['created_at'].dt.date

# set the cut off point
start_date = pd.to_datetime('2022-5-25')
end_date = pd.to_datetime('2023-5-1')

# cutting the date
df = df[(df['created_at'] >= start_date) & (df['created_at'] < end_date)]

C:\Users\koman\AppData\Local\Temp\ipykernel_12964\144948175.py:10: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  df = df[(df['created_at'] >= start_date) & (df['created_at'] < end_date)]


df.shape

### creating id columns

In [8]:
# sorting dataframe
df = df.sort_values(by='created_at')
df = df.reset_index(drop=True)

In [9]:
# create id column
df['id'] = np.arange(1, len(df) +1)

# swaping column
def swap_columns(df, id, url):
    col_list = list(df.columns)
    x, y = col_list.index(id), col_list.index(url)
    col_list[y], col_list[x] = col_list[x], col_list[y]
    df = df[col_list]
    return df
df = swap_columns(df, 'url', 'id')
df.tail()

,id,created_at,full_text,reply_count,retweet_count,user/advertiser_account_type,user/description,user/followers_count,user/friends_count,user/location,user/media_count,user/id,user/name,user/screen_name,user/url,user/verified,url
814,815,2022-12-02,Pertamina Uji Coba Beli Solar Subsidi Pakai QR...,0,1,promotable_user,"Let's Be Smart, part of Kapanlagi Youniverse W...",454421,62,Indonesia,75216,456426648,Merdeka.com,merdekadotcom,https://t.co/ZHg7lONlUI,True,https://twitter.com/merdekadotcom/status/15985...
815,816,2022-12-02,Btw kamu dukung tim sepak bola dari negara man...,0,0,promotable_user,"Official Account of Bright Gas, Inovasi produk...",10014,280,0,3444,703636439934701568,brightgas,brightgas_LPG,https://t.co/gsJs8Qnkdi,True,https://twitter.com/brightgas_LPG/status/15986...
816,817,2022-12-02,"@MyPertaminaID Halaah Min,,, ini tadi saya cob...",1,0,none,0,160,629,0,225,1662099961,PriyonoSlamet,Slametpriyono07,0,False,https://twitter.com/Slametpriyono07/status/159...
817,818,2022-12-02,Pertamina akhirnya menerapkan wajib scan QR co...,0,1,promotable_user,0,970716,3,0,81230,759692754985242625,kumparan,kumparan,https://t.co/ixWne53yta,True,https://twitter.com/kumparan/status/1598616854...
818,819,2022-12-02,@MyPertaminaID Royal enfield classic isi bengs...,3,0,none,Retail Trader - Temenan yok!,521,761,Megamendung - Jakarta,420,61730865,s_arifin,s_arifin,0,False,https://twitter.com/s_arifin/status/1598491994...


In [10]:
df_selected = df[['id', 'full_text', 'user/name', 'created_at']]

### 2. Re-Formating Text

In [11]:
# remove numbers
index = df_selected.index.values

for i in index:
    df_nbremoved = (df_selected[['id', 'full_text']]).values
    df_nbremoved[i][1] = re.sub(r"\d+", "", df_nbremoved[i][1])
    df_selected.at[i, 'full_text'] = df_nbremoved[i][1]
    
df_selected.tail(10)

,id,full_text,user/name,created_at
809,810,"@MyPertaminaID min, saya sudah coba regist di ...",Ibrahim Maulana,2022-12-02
810,811,@TimpalBali Aku pake myPertamina aja gadikais ...,kot,2022-12-02
811,812,"@MyPertaminaID pagi min, teman saya sudah daft...",ken,2022-12-02
812,813,Tunggu ya! \n\n#MakinAmanMakinCerdas #BrightGa...,brightgas,2022-12-02
813,814,@s_arifin @MyPertaminaID dibales mypertamina dong,Muthiah Azzahra,2022-12-02
814,815,Pertamina Uji Coba Beli Solar Subsidi Pakai QR...,Merdeka.com,2022-12-02
815,816,Btw kamu dukung tim sepak bola dari negara man...,brightgas,2022-12-02
816,817,"@MyPertaminaID Halaah Min,,, ini tadi saya cob...",PriyonoSlamet,2022-12-02
817,818,Pertamina akhirnya menerapkan wajib scan QR co...,kumparan,2022-12-02
818,819,@MyPertaminaID Royal enfield classic isi bengs...,s_arifin,2022-12-02


In [12]:
# remove punctuiation
index = df_selected.index.values

for i in index:
    df_pncremoved = (df_selected[['id', 'full_text']]).values
    df_pncremoved[i][1] = df_pncremoved[i][1].translate(str.maketrans("","",string.punctuation))
    df_selected.at[i, 'full_text'] = df_pncremoved[i][1]

df_selected.tail(10)

,id,full_text,user/name,created_at
809,810,MyPertaminaID min saya sudah coba regist di pt...,Ibrahim Maulana,2022-12-02
810,811,TimpalBali Aku pake myPertamina aja gadikais m...,kot,2022-12-02
811,812,MyPertaminaID pagi min teman saya sudah daftar...,ken,2022-12-02
812,813,Tunggu ya \n\nMakinAmanMakinCerdas BrightGas M...,brightgas,2022-12-02
813,814,sarifin MyPertaminaID dibales mypertamina dong,Muthiah Azzahra,2022-12-02
814,815,Pertamina Uji Coba Beli Solar Subsidi Pakai QR...,Merdeka.com,2022-12-02
815,816,Btw kamu dukung tim sepak bola dari negara man...,brightgas,2022-12-02
816,817,MyPertaminaID Halaah Min ini tadi saya coba pa...,PriyonoSlamet,2022-12-02
817,818,Pertamina akhirnya menerapkan wajib scan QR co...,kumparan,2022-12-02
818,819,MyPertaminaID Royal enfield classic isi bengsi...,s_arifin,2022-12-02


### 3. Stemming

In [13]:
# stemming 
fact = StemmerFactory()
stemmer = fact.create_stemmer()

index = df_selected.index.values

for i in index:
    df_stem = (df_selected[['id', 'full_text']]).values
    df_stem[i][1] = stemmer.stem(df_stem[i][1])
    df_selected.at[i, 'full_text'] = df_stem[i][1]
    
df_selected.tail(10)

,id,full_text,user/name,created_at
809,810,mypertaminaid min saya sudah coba regist di pt...,Ibrahim Maulana,2022-12-02
810,811,timpalbali aku pake mypertamina aja gadikais m...,kot,2022-12-02
811,812,mypertaminaid pagi min teman saya sudah daftar...,ken,2022-12-02
812,813,tunggu ya makinamanmakincerdas brightgas myper...,brightgas,2022-12-02
813,814,sarifin mypertaminaid dibales mypertamina dong,Muthiah Azzahra,2022-12-02
814,815,pertamina uji coba beli solar subsidi pakai qr...,Merdeka.com,2022-12-02
815,816,btw kamu dukung tim sepak bola dari negara man...,brightgas,2022-12-02
816,817,mypertaminaid halaah min ini tadi saya coba pa...,PriyonoSlamet,2022-12-02
817,818,pertamina akhir terap wajib scan qr code tiap ...,kumparan,2022-12-02
818,819,mypertaminaid royal enfield classic isi bengsi...,s_arifin,2022-12-02


### 4. Words Dictionary

In [14]:
# words count
kamus = df_selected.full_text.apply(lambda x: pd.value_counts(x.split(" "))).sum(axis = 0)

In [15]:
kamus_df = pd.DataFrame(kamus,columns=['total'])
text = kamus_df.index.values
kamus_df['word'] = text

# swaping column
def swap_columns(kamus_df, total, word):
    col_list = list(kamus_df.columns)
    x, y = col_list.index(total), col_list.index(word)
    col_list[y], col_list[x] = col_list[x], col_list[y]
    kamus_df = kamus_df[col_list]
    return kamus_df
kamus_df = swap_columns(kamus_df, 'total', 'word')

# reseting index
kamus_df = kamus_df.reset_index(drop=True)

# modify column
kamus_df = kamus_df.astype({'total':'int'})

# kamuskata = kamus_df[kamus_df['total']>10]
# kamuskata.to_excel('kamus_kata.xlsx', index=False)

In [16]:
# filter words <= 10
kamus_kurangdari_10 = kamus_df[kamus_df['total']<=10]
# kamus_kurangdari_10.to_excel('kamus_kurangdari_10.xlsx', index=False)

### 5. Tokenizing

In [17]:
# tokenizing
df_token = df_selected['full_text'].apply(word_tokenize)
df_selected['full_text'] = df_token

df_selected.tail(10)

C:\Users\koman\AppData\Local\Temp\ipykernel_12964\3987028371.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected['full_text'] = df_token


,id,full_text,user/name,created_at
809,810,"[mypertaminaid, min, saya, sudah, coba, regist...",Ibrahim Maulana,2022-12-02
810,811,"[timpalbali, aku, pake, mypertamina, aja, gadi...",kot,2022-12-02
811,812,"[mypertaminaid, pagi, min, teman, saya, sudah,...",ken,2022-12-02
812,813,"[tunggu, ya, makinamanmakincerdas, brightgas, ...",brightgas,2022-12-02
813,814,"[sarifin, mypertaminaid, dibales, mypertamina,...",Muthiah Azzahra,2022-12-02
814,815,"[pertamina, uji, coba, beli, solar, subsidi, p...",Merdeka.com,2022-12-02
815,816,"[btw, kamu, dukung, tim, sepak, bola, dari, ne...",brightgas,2022-12-02
816,817,"[mypertaminaid, halaah, min, ini, tadi, saya, ...",PriyonoSlamet,2022-12-02
817,818,"[pertamina, akhir, terap, wajib, scan, qr, cod...",kumparan,2022-12-02
818,819,"[mypertaminaid, royal, enfield, classic, isi, ...",s_arifin,2022-12-02


### 6. Stopwords Removal

In [18]:
# ignore important words
df_ignored = pd.read_excel(r"D:\work\project\SentimentAnalysis-MyPertamina\data\kamus kata\ignored_words.xlsx")
df_ignored = df_ignored['ignore'].tolist()
kamus_kurangdari_10['word'] = [' '.join([y for y in x.split() if y not in df_ignored])
                               for x in kamus_kurangdari_10['word']]

# adding stopwords
more_stopword_ind = pd.read_csv(r'D:\work\project\SentimentAnalysis-MyPertamina\data\stopwords\stopwordbahasa.csv')
more_stopword = kamus_kurangdari_10['word'].tolist()

# merge stopwords
stopwords = more_stopword + more_stopword_ind['stop'].tolist()

# removing stopwords
index = df_selected.index.values
for i in index:
    df_stoprmv = (df_selected[['id', 'full_text']]).values
    df_stoprmv[i][1] = [x for x in df_stoprmv[i][1] if x not in stopwords]
    df_selected.at[i, 'full_text'] = df_stoprmv[i][1]

df_selected.head(5)

C:\Users\koman\AppData\Local\Temp\ipykernel_12964\4248885813.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kamus_kurangdari_10['word'] = [' '.join([y for y in x.split() if y not in df_ignored])


,id,full_text,user/name,created_at
0,1,"[subsidi, bbm, tepat, sasar, beli, pertalite, ...",Dina,2022-05-28
1,2,"[tuju, bbm, subsidi, masyarakat, hak, subsidi,...",Adam,2022-05-28
2,3,"[tuju, bbm, subsidi, masyarakat, hak, subsidi,...",Lahrian Syahilla,2022-05-29
3,4,"[solusi, tepat, subsidi, bbm, tepat, sasar, be...",My time :),2022-05-29
4,5,"[subsidi, bbm, tepat, sasar, beli, pertalite, ...",Riri! fb ya,2022-05-29


### 7. Labeling

In [19]:
# labeling
listNegatif = pd.read_excel(r'D:\work\project\SentimentAnalysis-MyPertamina\data\kamus kata\negatif.xlsx')
listNegatif = listNegatif['negatif'].tolist()
listPositif = pd.read_excel(r'D:\work\project\SentimentAnalysis-MyPertamina\data\kamus kata\positif.xlsx')
listPositif = listPositif['positif'].tolist()

df_selected["label"] = df_selected['full_text'].apply(lambda x: 'negatif' if any(i in x for i in listNegatif) 
                                                      else 'positif' 
                                                      if any(i in x for i in listPositif) else 'netral')

C:\Users\koman\AppData\Local\Temp\ipykernel_12964\889802031.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected["label"] = df_selected['full_text'].apply(lambda x: 'negatif' if any(i in x for i in listNegatif)


In [20]:
df_positif = df_selected[df_selected['label']=='positif']
df_negatif = df_selected[df_selected['label']=='negatif']
df_netral = df_selected[df_selected['label']=='netral']

print(f'positif {df_positif.shape} record \n'
      f'negatif {df_negatif.shape} record \n'
      f'netral {df_netral.shape} record')

positif (278, 5) record 
negatif (231, 5) record 
netral (310, 5) record


In [21]:
data = df_selected[['id', 'full_text', 'user/name', 'created_at','label']]
data.tail(10)

,id,full_text,user/name,created_at,label
809,810,"[mypertaminaid, coba, mypertamina]",Ibrahim Maulana,2022-12-02,netral
810,811,"[pake, mypertamina]",kot,2022-12-02,netral
811,812,"[mypertaminaid, daftar, mypertamina, salah, so...",ken,2022-12-02,negatif
812,813,"[tunggu, mypertamina, call]",brightgas,2022-12-02,negatif
813,814,"[mypertaminaid, mypertamina]",Muthiah Azzahra,2022-12-02,netral
814,815,"[pertamina, uji, coba, beli, solar, subsidi, p...",Merdeka.com,2022-12-02,netral
815,816,"[dukung, negara, mypertamina, call]",brightgas,2022-12-02,positif
816,817,"[mypertaminaid, halaah, coba, pakai, aplikasi,...",PriyonoSlamet,2022-12-02,negatif
817,818,"[pertamina, terap, wajib, qr, beli, bbm, solar...",kumparan,2022-12-02,netral
818,819,"[mypertaminaid, isi, pake, pertamax, bayar, my...",s_arifin,2022-12-02,positif


In [22]:
index = data.index.values

for i in index:
    increment = i+1
    seleksi = data[data['id']==increment]
    print(seleksi[['id','full_text']].values)


[[1
  list(['subsidi', 'bbm', 'tepat', 'sasar', 'beli', 'pertalite', 'atur', 'pakai', 'aplikasi', 'mypertamina', 'solusi', 'tepat', 'dukung'])]]
[[2
  list(['tuju', 'bbm', 'subsidi', 'masyarakat', 'hak', 'subsidi', 'bbm', 'tepat', 'sasar', 'beli', 'pertalite', 'atur', 'pakai', 'aplikasi', 'mypertamina'])]]
[[3
  list(['tuju', 'bbm', 'subsidi', 'masyarakat', 'hak', 'subsidi', 'bbm', 'tepat', 'sasar', 'beli', 'pertalite', 'atur', 'pakai', 'aplikasi', 'mypertamina'])]]
[[4
  list(['solusi', 'tepat', 'subsidi', 'bbm', 'tepat', 'sasar', 'beli', 'pertalite', 'atur', 'pakai', 'aplikasi', 'mypertamina'])]]
[[5
  list(['subsidi', 'bbm', 'tepat', 'sasar', 'beli', 'pertalite', 'atur', 'pakai', 'aplikasi', 'mypertamina', 'solusi', 'upaya', 'tepat'])]]
[[6
  list(['tepat', 'sasar', 'beli', 'pertalite', 'solar', 'pakai', 'mypertamina'])]]
[[7
  list(['perintah', 'rencana', 'aplikasi', 'mypertamina', 'salur', 'bahan', 'bakar', 'minyak', 'bbm', 'subsidi', 'solar', 'pertalite', 'tepat', 'sasar'])]]
[[8

# C. Spliting and TF-IDF

In [23]:
# split dataset 
X_train, X_test, y_train, y_test = train_test_split(  
                                                data['full_text'],  
                                                data['label'],   
                                                test_size=0.4,
                                                random_state=5)

In [24]:
print('y_test = ',y_test.shape)
print('X_test = ',X_test.shape)

print('y_train = ',y_train.shape)
print('X_train = ',X_train.shape)

y_test =  (328,)
X_test =  (328,)
y_train =  (491,)
X_train =  (491,)


In [25]:
# tf-idf
input_corrected_train = [" ".join(x) for x in X_train]
input_corrected_test = [" ".join(x) for x in X_test]


vectorizer = TfidfVectorizer()
train_tfidf = vectorizer.fit_transform(input_corrected_train)
test_tfidf = vectorizer.transform(input_corrected_test)

# convert to dataframe
train_tfidf = pd.DataFrame(train_tfidf.toarray(), columns=vectorizer.get_feature_names_out())
test_tfidf = pd.DataFrame(test_tfidf.toarray(), columns=vectorizer.get_feature_names_out())

# res = pd.concat([df, df1], axis=1)

In [26]:
train_tfidf

,akun,aman,anjink,antre,antusiasme,aplikasi,atur,awas,bahan,bakar,...,tuju,tunggu,uang,uji,untung,upaya,utk,via,wajib,warga
0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.000000,0.0,0.0,0.488576,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.249265,0.0,0.0,0.000000,0.000000,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.165308,0.0,0.0,0.000000,0.000000,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
486,0.0,0.0,0.0,0.0,0.0,0.225523,0.0,0.0,0.000000,0.000000,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
487,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.245743,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
488,0.0,0.0,0.0,0.0,0.0,0.186028,0.0,0.0,0.000000,0.000000,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
489,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


In [27]:
test_tfidf

,akun,aman,anjink,antre,antusiasme,aplikasi,atur,awas,bahan,bakar,...,tuju,tunggu,uang,uji,untung,upaya,utk,via,wajib,warga
0,0.0,0.0,0.0,0.0,0.0,0.385506,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.317916,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.360275,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
324,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.345592,0.0,0.000000,0.0,0.0
325,0.0,0.0,0.0,0.0,0.0,0.256858,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
326,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.516946,0.0,0.0


# D. SMOTE
###  Synthetic Minority Over-sampling Technique

In [28]:
# y_train before balancing
counter = Counter(y_train)
print(counter)

Counter({'netral': 198, 'positif': 158, 'negatif': 135})


In [29]:
# balancing training set
oversample = SMOTE()
x_sm, y_sm = oversample.fit_resample(train_tfidf, y_train)

In [30]:
# y_train after balancing
counter = Counter(y_sm)
print(counter)

Counter({'negatif': 198, 'netral': 198, 'positif': 198})


# E. NBC

In [31]:
# model dengan SMOTE
clfsm = MultinomialNB()
clfsm.fit(x_sm, y_sm)

# model tanpa SMOTE
clf = MultinomialNB()
clf.fit(train_tfidf, y_train)

MultinomialNB()

In [32]:
# prediksi tanpa SMOTE
y_pred = clf.predict(test_tfidf)

# prediksi dengan SMOTE
y_predsm = clfsm.predict(test_tfidf)

In [33]:
# prediksi tanpa smote
print("hasil prediksi tanpa mengimplementasikan smote: \n")
print(classification_report(y_test, y_pred))

hasil prediksi tanpa mengimplementasikan smote: 

              precision    recall  f1-score   support

     negatif       0.94      0.50      0.65        96
      netral       0.66      0.96      0.78       112
     positif       0.84      0.82      0.83       120

    accuracy                           0.77       328
   macro avg       0.82      0.76      0.76       328
weighted avg       0.81      0.77      0.76       328



In [34]:
# prediksi dengan SMOTE
print("hasil prediksi dengan mengimplementasikan smote: \n")
print(classification_report(y_test, y_predsm))

hasil prediksi dengan mengimplementasikan smote: 

              precision    recall  f1-score   support

     negatif       0.85      0.71      0.77        96
      netral       0.74      0.86      0.79       112
     positif       0.84      0.82      0.83       120

    accuracy                           0.80       328
   macro avg       0.81      0.80      0.80       328
weighted avg       0.81      0.80      0.80       328



In [35]:
# df.to_excel('df_clean.xlsx', index=False)